In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Auto-sklearn
Auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator.

For more information about the framework, please visit the documentation [here](https://automl.github.io/auto-sklearn/master/).

## What you will need to run the code

In order to run this code, we are going to first install Auto-sklearn using pip. For more instructions on how to install Auto-sklearn, for example using conda, please check [this](https://automl.github.io/auto-sklearn/master/installation.html).

In [1]:
# For google colab, we need to force the installation of the a scikit learn
# version
#!pip uninstall -y scikit-learn
#!pip install 'scikit-learn>=0.24.0'
#!pip uninstall -y distributed
#!pip install 'distributed==2.30.1'
!pip install -U numpy 
!pip install auto-sklearn
import autosklearn

print("Using Auto-sklearn version {}".format(autosklearn.__version__))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Using Auto-sklearn version 0.15.0


## First Step: Load data

Auto-sklearn can work with multiple input data formats (python lists, numpy arrays, sparse arrays and pandas data-frames).  

For this example we are going to be using the [credit-g dataset](https://www.openml.org/d/31) which is a binary classification problem. This means that we have to find an estimator that is able to predict between 2 categories, *'bad'* and *'good'*.


In [2]:
import sklearn.datasets
import sklearn.model_selection

# We fetch the data using the openml.org
X, y = sklearn.datasets.fetch_openml(data_id=31, return_X_y=True, as_frame=True)

# Split the data into train and test
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, test_size=0.4, random_state=42
)

X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 24 to 102
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   checking_status         600 non-null    category
 1   duration                600 non-null    float64 
 2   credit_history          600 non-null    category
 3   purpose                 600 non-null    category
 4   credit_amount           600 non-null    float64 
 5   savings_status          600 non-null    category
 6   employment              600 non-null    category
 7   installment_commitment  600 non-null    float64 
 8   personal_status         600 non-null    category
 9   other_parties           600 non-null    category
 10  residence_since         600 non-null    float64 
 11  property_magnitude      600 non-null    category
 12  age                     600 non-null    float64 
 13  other_payment_plans     600 non-null    category
 14  housing                 6

## Second Step: Manually build a pipeline

For this tutorial, we are going to implement some traditional machine learning models ([GradientBoosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html), [Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html), [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html)) using [scikit-learn](https://scikit-learn.org/stable/index.html). Then we are going to show how we can achieve an even better performance than these traditional models, by using Auto-Sklearn.


### C-Support Vector Classification



In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


# Create the estimator using the default parameters from the library
estimator_svc = SVC(
    C=1.0, kernel='rbf', gamma='scale', shrinking=True, tol=1e-3,
    cache_size=200, verbose=False, max_iter=-1, random_state=42
)

# build and fit the pipeline
categorical_columns = [col for col in X_train.columns 
                       if X[col].dtype.name == 'category']
encoder = ColumnTransformer(transformers = [
  ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
], remainder='passthrough')
pipeline_svc = Pipeline([
  ('encoder', encoder),
  ('scaler', StandardScaler()),                   
  ('svc', estimator_svc),
])
pipeline_svc.fit(X_train, y_train)

# Score the model
prediction = pipeline_svc.predict(X_test)
performance_svc = accuracy_score(y_test, prediction)
print(f"SVC performance is {performance_svc}")

SVC performance is 0.7675


### GradientBoostingClassifier

In [4]:
from sklearn.ensemble import GradientBoostingClassifier

# Create the estimator using default parameters from the library
estimator_gradboost = GradientBoostingClassifier(
    learning_rate=0.1, n_estimators=100, subsample=1.0, 
    criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0,
    random_state=42)

# Translate the categorical columns to 
# a numerical value
categorical_columns = [col for col in X_train.columns 
                       if X[col].dtype.name == 'category']
encoder = ColumnTransformer(transformers = [
  ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
], remainder='passthrough')


# Build and fit the pipeline
pipeline_gradboost = Pipeline([
  ('encoder', encoder),
  ('gradboost', estimator_gradboost),
])
pipeline_gradboost.fit(X_train, y_train)

# Score the model
prediction = pipeline_gradboost.predict(X_test)
performance_gradboost = accuracy_score(y_test, prediction)
print(f"GradientBooster performance is {performance_gradboost}")

GradientBooster performance is 0.735


### Decision tree classifier

In [5]:
from sklearn.tree import DecisionTreeClassifier

# Create the estimator using the default parameters from the library
estimator_tree = DecisionTreeClassifier(random_state=42)

# build and fit the pipeline
encoder = ColumnTransformer(transformers = [
  ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
], remainder='passthrough')
pipeline_tree = Pipeline([
  ('encoder', encoder),
  ('DecisionTree', estimator_tree),
])
pipeline_tree.fit(X_train, y_train)

# Predict on the training data
prediction = pipeline_tree.predict(X_test)

# Evaluate the performance of the model
performance_tree = accuracy_score(y_test, prediction)
print(f"Decision Tree performance is {performance_tree}")

Decision Tree performance is 0.7075


# Third Step: Use Auto-sklearn as a drop-in-replacement


In [6]:
import autosklearn.classification

# Create and train the estimator 
estimator_askl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=300, 
    seed=42, 
    resampling_strategy='cv',
    n_jobs=1,
)
estimator_askl.fit(X_train, y_train)

# Score the model
prediction = estimator_askl.predict(X_test)
performance_askl = accuracy_score(y_test, prediction)
print(f"Auto-Sklearn Classifier performance is {performance_askl}")

/usr/local/lib/python3.10/dist-packages/autosklearn/data/target_validator.py:187: UserWarning: Fitting transformer with a pandas series which has the dtype category. Inverse transform may not be able preserve dtype when converting to np.ndarray
  warnings.warn(


Auto-Sklearn Classifier performance is 0.785


In [7]:
import pandas as pd
from google.colab import data_table

# By using Auto-Sklearn on can achieve a better performance!
data_table.DataTable(
pd.DataFrame(
    [
     {'Model': 'Auto-Sklearn Classifier', 'Accuracy': performance_askl},
     {'Model': 'GradientBoosting', 'Accuracy': performance_gradboost},
     {'Model': 'Decision Tree Classifier', 'Accuracy': performance_tree},
     {'Model': 'Support Vector Classifier', 'Accuracy': performance_svc},
     ]
))

,Model,Accuracy
0,Auto-Sklearn Classifier,0.7850
1,GradientBoosting,0.7350
2,Decision Tree Classifier,0.7075
3,Support Vector Classifier,0.7675
